# Getting Started with boto on AWS

In the first step we need to import all required packages and modules into the Python Path

In [1]:
# System Libraries
import sys, os
sys.path.append("..")
import pandas as pd

## logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
 
import os
import json

import boto3
from botocore.config import Config
boto3.setup_default_session(profile_name='blockchain')

# 3. EC2

In [2]:
ec2_client = boto3.resource('ec2', region_name='eu-central-1', config=Config(proxies={'https': 'proxy.ccc.eu-central-1.aws.cloud.bmw:8080',
                                                                                       'http': 'proxy.ccc.eu-central-1.aws.cloud.bmw:8080'}
                                                                                     ))
ec2_description =  {
    "number_vms" : 1,
    "ec2_ssh_username":"ubuntu",
    "ec2_ssh_keyname":"blockchain",
    "ec2_ssh_keyfile":os.path.join(os.environ["HOME"], ".aws/blockchain/blockchain"),    
    "ec2_vpc_id" : "vpc-0372565c41602fcad",   
    "ec2_subnet_id": "subnet-075e7cd32d7c07a67",
    "ec2_image_id" : "ami-090f10efc254eaf55", 
    "ec2_security_group": "sg-06af921573efee0f9",
    "ec2_instance_type" : "t2.nano",
    "ec2_name": "blockchain-1", 
    "type":"ec2"
}

ec2_instances = ec2_client.create_instances(ImageId=ec2_description["ec2_image_id"],
                                            InstanceType=ec2_description["ec2_instance_type"],
                                            KeyName=ec2_description["ec2_ssh_keyname"],
                                            SubnetId=ec2_description["ec2_subnet_id"],
                                            SecurityGroupIds=[ec2_description["ec2_security_group"]],
                                            TagSpecifications=[{'ResourceType': 'instance',
                                                                'Tags': [{"Key":"Name", 
                                                                          "Value":ec2_description["ec2_name"]}]}],
                                            MinCount=ec2_description["number_vms"], 
                                            MaxCount=ec2_description["number_vms"])

ConnectTimeoutError: Connect timeout on endpoint URL: "https://sts.amazonaws.com/"

In [ ]:
ips=[]
for i in ec2_instances:
    i.wait_until_running()
    i.load()
    print("ID: {}, State: {}, IP: {} ".format(i.id, i.state['Name'], i.private_ip_address))
    ips.append(i.private_ip_address)

In [ ]:
!ssh -i {ec_description["kec2_ssh_keyfile"]} -l {ec_description["ec2_ssh_username"]} {ips[1]} /bin/hostname

In [ ]:
for i in ec2_instances:
    i.terminate()

In [ ]:
for i in ec2_client.instances.all():
    print(i.state)